In [1]:
from PyPDF2 import PdfReader, PdfWriter
import numpy as np
import cohere
import re
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
def extract_pages(input_pdf, output_pdf, pages_to_extract):
    # Create a PDF reader and writer
    reader = PdfReader(input_pdf)
    writer = PdfWriter()
    
    # Extract the specified pages
    for page_num in pages_to_extract:
        writer.add_page(reader.pages[page_num])

    # Write the extracted pages to a new PDF
    with open(output_pdf, 'wb') as output_file:
        writer.write(output_file)

    print(f"New PDF created: {output_pdf}")


# input_pdf = r'D:\Programming\AI\Basics\AMNIL Tech\Chat With Docs\energy.pdf' 
# output_pdf = 'sample.pdf'       
# pages_to_extract = [0,1]           

# extract_pages(input_pdf, output_pdf, pages_to_extract)


In [2]:
pdf_path = 'BERT.pdf'
reader = PdfReader(pdf_path)
page = reader.pages[0]

In [3]:
parts = []
def visitor_body(text, cm, tm, fontDict, fontSize):
    y = tm[5]        
    if y > 50 and y < 800:
        print(f"Text: {repr(text)}, Font Size: {fontSize}")
        parts.append(text)
page.extract_text(visitor_text=visitor_body)
text_body = "".join(parts)

Text: 'BERT: Pre-training of Deep Bidirectional Transformers for\n', Font Size: 14.3462
Text: 'Language Understanding', Font Size: 14.3462
Text: '\n', Font Size: 11.9552
Text: 'Jacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova', Font Size: 11.9552
Text: '\n', Font Size: 11.9552
Text: 'Google AI Language', Font Size: 11.9552
Text: '\n', Font Size: 11.9552
Text: 'f', Font Size: 11.9552
Text: 'jacobdevlin,mingweichang,kentonl,kristout', Font Size: 11.9552
Text: ' g', Font Size: 11.9552
Text: '@google.com', Font Size: 11.9552
Text: '\n', Font Size: 11.9552
Text: 'Abstract', Font Size: 11.9552
Text: '\n', Font Size: 9.9626
Text: 'We introduce a new language representa-\n', Font Size: 9.9626
Text: 'tion model called', Font Size: 9.9626
Text: ' BERT', Font Size: 9.9626
Text: ' , which stands for', Font Size: 9.9626
Text: '\n', Font Size: 9.9626
Text: 'B', Font Size: 9.9626
Text: 'idirectional', Font Size: 9.9626
Text: ' E', Font Size: 9.9626
Text: 'ncoder', Font Size: 9.9626
Text: ' R

In [4]:
def chunk_by_sentence(text, max_length):
    # Regular expression to match sentence-ending punctuation: . ? !
    sentence_endings = re.compile(r'([.!?])')
    text = re.sub(r'\s+', ' ', text).strip()
    # Split the text into sentences based on the sentence-ending punctuation
    sentences = sentence_endings.split(text)
    
    # Recombine punctuation with the sentences
    sentences = [sentences[i] + (sentences[i+1] if i+1 < len(sentences) else '') for i in range(0, len(sentences), 2)]
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        # If adding the sentence exceeds max_length, start a new chunk
        if len(current_chunk) + len(sentence) > max_length:
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = sentence.strip()
        else:
            current_chunk += sentence  # Add the sentence to the current chunk

    if current_chunk:  # Append the last chunk
        chunks.append(current_chunk)
    
    return chunks

In [5]:
chunks = chunk_by_sentence(text_body,500)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")

Chunk 1:
BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding Jacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova Google AI Language fjacobdevlin,mingweichang,kentonl,kristout g@google.com Abstract We introduce a new language representa- tion model called BERT , which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language repre- sentation models (Peters et al., 2018a; Rad- ford et al.

Chunk 2:
, 2018), BERT is designed to pre- train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a re- sult, the pre-trained BERT model can be ﬁne- tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task- speciﬁc architecture modiﬁcations. BERT is conceptually simple and empirically powerful.

Chunk 3:
It obtains new state-of-the-art 

In [6]:
chunks

['BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding Jacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova Google AI Language fjacobdevlin,mingweichang,kentonl,kristout g@google.com Abstract We introduce a new language representa- tion model called BERT , which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language repre- sentation models (Peters et al., 2018a; Rad- ford et al.',
 ', 2018), BERT is designed to pre- train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a re- sult, the pre-trained BERT model can be ﬁne- tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task- speciﬁc architecture modiﬁcations. BERT is conceptually simple and empirically powerful.',
 'It obtains new state-of-the-art re- sults on eleven

In [7]:
def calculate_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [8]:
COHERE_KEY = os.getenv('COHERE_KEY')
co = cohere.Client(COHERE_KEY)

In [9]:
model = "embed-english-v3.0"
input_type = "search_document"

chunk_embeddings = co.embed(
    texts = chunks,
    model = model,
    input_type = input_type,
    embedding_types=['float']
)

In [10]:
input_type = 'search_query'
query = 'What are the major limitations of standard lanugage models?'
query_embedding = co.embed(
    texts = [query],
    model = model,
    input_type = input_type,
    embedding_types= ['float']
)

In [11]:
chunk_embedding_floats = chunk_embeddings.embeddings.float
query_embedding_float = query_embedding.embeddings.float[0]
for chunk_embedding_float in chunk_embedding_floats:
    print(calculate_similarity(chunk_embedding_float,query_embedding_float))

0.14988067304180056
0.1956473193532332
0.17145194664191885
0.17134736578339163
0.19394368632370673
0.19504123330431916
0.28432602192053075
0.4630081800950562
0.29434924040318294
0.32752604537078445


In [12]:
chunks[7]

'The ma- jor limitation is that standard language models are unidirectional, and this limits the choice of archi- tectures that can be used during pre-training. For example, in OpenAI GPT, the authors use a left-to- right architecture, where every token can only at- tend to previous tokens in the self-attention layers of the Transformer (Vaswani et al., 2017).'

In [15]:
# Input prompt for the model
prompt = """
Use only the context provided to answer the following question. If you don't know the answer, reply that you are unsure.
Context: {chunks[7]} and {chunks[9]}
Question: {query}
"""

# Generate text
response = co.generate(
    model="command",
    prompt=prompt,
    max_tokens=200,  
    temperature=0.7,  
)

In [17]:
chunks[7]

'The ma- jor limitation is that standard language models are unidirectional, and this limits the choice of archi- tectures that can be used during pre-training. For example, in OpenAI GPT, the authors use a left-to- right architecture, where every token can only at- tend to previous tokens in the self-attention layers of the Transformer (Vaswani et al., 2017).'

In [18]:
chunks[9]

'BERT alleviates the previously mentioned unidi- rectionality constraint by using a “masked lan- guage model” (MLM) pre-training objective, in- spired by the Cloze task (Taylor, 1953). The masked language model randomly masks some of the tokens from the input, and the objective is to predict the original vocabulary id of the maskedarXiv:1810.04805v2 [cs.CL] 24 May 2019'

In [16]:
# Access the generated text
print(response.generations[0].text)

 The major limitation with standard language models is that they are typically unidirectional, which limits the kinds of architectures that can be used during pretraining.  OpenAI GPT uses a left-to-right architecture, where each token can only attend to previous tokens in the self-attention layers of the Transformer.  BERT is an example of an architecture that works around this limitation by using a masked language model pretraining objective inspired by the Cloze task, where the model randomly masks input tokens and predicts the masked tokens, increasing bidirectionality.  This allows BERT to outperform previous models on a number of tasks.  Overall, bidirectionality is a key aspect of more advanced language models that greatly improves their performance.  Without this limitation, the potential of the models increases. 


In [74]:
# phrases = ["i love soup", "soup is my favorite", "london is far away"]
# model = "embed-english-v3.0"
# input_type = "search_query"
# res = co.embed(
#     texts=phrases,
#     model=model,
#     input_type=input_type,
#     embedding_types=["float"],
# )
# (soup1, soup2, london) = res.embeddings.float
# calculate_similarity(soup1, soup2)  # 0.85 
# calculate_similarity(soup1, london) # 0.164789